In [1]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.vis_utils import plot_model
from keras.applications.vgg19 import decode_predictions
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow
from time import time

import pandas

import pickle as pk

from keras.wrappers.scikit_learn import KerasClassifier

seed = 7

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
np.random.seed(seed)

In [3]:
data_dir = '/data/oxford102/train'
img_width, img_height = 256, 256
batch_size = 32
epochs = 100
nr_categories = 102

In [4]:
img_width, img_height = 256, 256
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
nb_train_samples = 4604
nb_validation_samples = 1094 

In [5]:
input_processor = keras.applications.vgg19.preprocess_input

In [6]:
train_val_datagen = ImageDataGenerator(preprocessing_function=input_processor, 
                                           validation_split=0.2)

train_val_datagen_aug = ImageDataGenerator(
        #rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=input_processor,
        validation_split=0.2)

In [7]:

    
train_generator = train_val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

validation_generator = train_val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.


In [8]:
#from keras import regularizers

k.set_learning_phase(0)

network_name = "vgg19"
img_width, img_height = (256, 256)
if network_name == "vgg16":
    base_model = keras.applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
elif network_name == "vgg19":
    base_model = keras.applications.vgg19.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
else:
    raise Exception("check your network name")

for layer in base_model.layers[:]:
    layer.trainable = False

    #Adding custom Layers 
k.set_learning_phase(1)
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu", 
          #kernel_regularizer=regularizers.l2(0.01),
         #       activity_regularizer=regularizers.l1(0.001)
         )(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x, training=True)
#x = Dense(102, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

_model = Model(input = base_model.input, output = predictions)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:30: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [9]:
_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [10]:
# _model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [11]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 256,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
17/17 [==============================] - 17s 1s/step - loss: 5.0826 - acc: 0.0239 - val_loss: 4.3120 - val_acc: 0.0859

Epoch 00001: val_acc improved from -inf to 0.08594, saving model to /data/oxford102/experiments/1543113644.933369/vgg19_1543113644.933369.h5
Epoch 2/100
17/17 [==============================] - 10s 565ms/step - loss: 4.0067 - acc: 0.1618 - val_loss: 3.3736 - val_acc: 0.2891

Epoch 00002: val_acc improved from 0.08594 to 0.28906, saving model to /data/oxford102/experiments/1543113644.933369/vgg19_1543113644.933369.h5
Epoch 3/100
17/17 [==============================] - 10s 570ms/step - loss: 3.2117 - acc: 0.2868 - val_loss: 3.1293 - val_acc: 0.2891

Epoch 00003: val_acc did not improve from 0.28906
Epoch 4/100
17/17 [==============================] - 10s 569ms/step - loss: 2.8625 - acc: 0.3640 - val_loss: 2.6578 - val_acc: 0.4297

Epoch 00004: val_acc improved from 0.28906 to 0.42969, saving model to /data/oxford102/experiments/1543113644.933369/vgg19_15431

In [12]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094

params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
35/35 [==============================] - 21s 591ms/step - loss: 0.6700 - acc: 0.9134 - val_loss: 1.4278 - val_acc: 0.6797

Epoch 00001: val_acc improved from -inf to 0.67969, saving model to /data/oxford102/experiments/1543113850.0769787/vgg19_1543113850.0769787.h5
Epoch 2/100
35/35 [==============================] - 19s 556ms/step - loss: 0.7015 - acc: 0.8830 - val_loss: 1.4974 - val_acc: 0.6641

Epoch 00002: val_acc did not improve from 0.67969
Epoch 3/100
35/35 [==============================] - 20s 559ms/step - loss: 0.7379 - acc: 0.8777 - val_loss: 1.3917 - val_acc: 0.7031

Epoch 00003: val_acc improved from 0.67969 to 0.70312, saving model to /data/oxford102/experiments/1543113850.0769787/vgg19_1543113850.0769787.h5
Epoch 4/100
35/35 [==============================] - 20s 557ms/step - loss: 0.6602 - acc: 0.8936 - val_loss: 1.3292 - val_acc: 0.7070

Epoch 00004: val_acc improved from 0.70312 to 0.70703, saving model to /data/oxford102/experiments/1543113850.0769787/vgg

In [13]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
71/71 [==============================] - 40s 568ms/step - loss: 0.2747 - acc: 0.9798 - val_loss: 1.1395 - val_acc: 0.7537

Epoch 00001: val_acc improved from -inf to 0.75368, saving model to /data/oxford102/experiments/1543114085.991548/vgg19_1543114085.991548.h5
Epoch 2/100
71/71 [==============================] - 40s 557ms/step - loss: 0.2769 - acc: 0.9801 - val_loss: 1.1201 - val_acc: 0.7629

Epoch 00002: val_acc improved from 0.75368 to 0.76287, saving model to /data/oxford102/experiments/1543114085.991548/vgg19_1543114085.991548.h5
Epoch 3/100
71/71 [==============================] - 40s 560ms/step - loss: 0.1837 - acc: 0.9921 - val_loss: 1.0271 - val_acc: 0.7702

Epoch 00003: val_acc improved from 0.76287 to 0.77022, saving model to /data/oxford102/experiments/1543114085.991548/vgg19_1543114085.991548.h5
Epoch 4/100
71/71 [==============================] - 40s 560ms/step - loss: 0.1932 - acc: 0.9885 - val_loss: 1.1312 - val_acc: 0.7408

Epoch 00004: val_acc did not im

In [14]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
143/143 [==============================] - 80s 559ms/step - loss: 0.0670 - acc: 0.9993 - val_loss: 1.0375 - val_acc: 0.7537

Epoch 00001: val_acc improved from -inf to 0.75368, saving model to /data/oxford102/experiments/1543114611.679193/vgg19_1543114611.679193.h5
Epoch 2/100
143/143 [==============================] - 80s 557ms/step - loss: 0.0575 - acc: 0.9993 - val_loss: 0.9994 - val_acc: 0.7711

Epoch 00002: val_acc improved from 0.75368 to 0.77114, saving model to /data/oxford102/experiments/1543114611.679193/vgg19_1543114611.679193.h5
Epoch 3/100
143/143 [==============================] - 80s 557ms/step - loss: 0.0521 - acc: 0.9998 - val_loss: 1.0095 - val_acc: 0.7656

Epoch 00003: val_acc did not improve from 0.77114
Epoch 4/100
143/143 [==============================] - 80s 557ms/step - loss: 0.0441 - acc: 0.9998 - val_loss: 0.9883 - val_acc: 0.7785

Epoch 00004: val_acc improved from 0.77114 to 0.77849, saving model to /data/oxford102/experiments/1543114611.679193/

# FINETUNE TOP LAYERS

In [ ]:
for i, layer in enumerate(base_model.layers[11:]):
    print(i, layer.name)
    layer.trainable = True

In [ ]:
model_name = "vgg19"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg19',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 16,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params